In [1]:
## Load modules
import sys
if "../.." not in sys.path: sys.path.insert(0, "../..")
if ".." not in sys.path: sys.path.insert(0, "..")

#Base
from os import path
import numpy as np
import xarray as xr
import pandas as pd
import scipy.stats as st
from tqdm.notebook import trange, tqdm
import time

#Plot
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

#V1DD
from experiment.v1dd_em_client import V1DDEMClient
from experiment.v1dd_client import V1DDClient
from experiment.v1dd_ophys_session import V1DDOPhysSession
from stimulus_analysis.stimulus_analysis import StimulusAnalysis
from stimulus_analysis.drifting_gratings import DriftingGratings
from stimulus_analysis.natural_scenes import NaturalScenes
from stimulus_analysis import fit_utils
import analysis_tools as at

#User
import stimulus_analysis.decoding as dc

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [4]:
## Load data
base_folder = r"\\allen\programs\mindscope\workgroups\surround\v1dd_in_vivo_new_segmentation\data" #Server
base_folder = "/home/dwyrick/data/v1dd_in_vivo_new_segmentation" # Local on Essun for golden mouse
client = V1DDClient(base_folder)


## Load em client
em_client = V1DDEMClient()

# ##===== Read in drifting grating cell metrics =====##
# metrics_df = pd.read_csv("../../dataframes/v1dd_metrics.csv", index_col=0, low_memory=False)
# metrics_df = metrics_df[(metrics_df.is_valid) & (metrics_df.mouse == 409828) & (metrics_df.column == 1) & (metrics_df.volume == '3')]



HTTPError: 401 Client Error: UNAUTHORIZED for url: https://globalv1.em.brain.allentech.org/info/api/v2/datastack/full/v1dd content: b'{\n  "error": "invalid_token", \n  "message": "Unauthorized - Token is Invalid or Expired"\n}\n'

In [3]:
## Read correspondence table from Russel
correspondence_df = pd.read_csv("../../dataframes/20230121_correspondence_table.tsv", sep='\t',index_col=4, low_memory=False)

print("Coregistered cells:")
em_client.get_2p_corresponded_table().groupby(["axon_proofreading_complete", "dendrite_proofreading_complete"]).size().sort_index(ascending=False)

Coregistered cells:


NameError: name 'em_client' is not defined

In [9]:
correspondence_df

,nucleus_point_nm,ophys_column,ophys_volume,ophys_plane_name,ophys_roi_point,correspondence_residual_nm,ctrl_pt_distance_um,fit_expectation_um,corr_method
ophys_roi_name,,,,,,,,,
roi_0000,"[899074, 635466, 83520]",1,3,plane0,"[359.47229551451187, 46.36059806508356, 0.0]",4849.713802,6.498279,1.708822,fit_search_v0.0.1
roi_0001,"[787485, 606599, 174105]",1,3,plane0,"[188.08246225319394, 165.51335656213706, 0.0]",6490.590771,28.217650,1.646525,fit_search_v0.0.1
roi_0002,"[902255, 558254, 308655]",1,3,plane0,"[315.91832229580575, 343.1931567328918, 0.0]",10463.777854,31.819328,3.149323,fit_search_v0.0.1
roi_0003,"[846383, 592942, 209025]",1,3,plane0,"[266.65572139303487, 206.9223880597015, 0.0]",2271.853481,19.021159,3.207205,fit_search_v0.0.1
roi_0004,"[901324, 636320, 75465]",1,3,plane0,"[356.6945273631841, 32.32935323383085, 0.0]",3436.624502,15.458198,1.923929,fit_search_v0.0.1
...,...,...,...,...,...,...,...,...,...
roi_1143,"[762963, 672404, 174510]",1,3,plane5,"[149.16407599309153, 168.00690846286702, 5.0]",4588.167644,46.487899,6.287323,fit_search_v0.0.1
roi_1144,"[917465, 671783, 228465]",1,3,plane5,"[352.74074074074076, 231.36296296296297, 5.0]",7320.071876,35.853716,6.524178,fit_search_v0.0.1
roi_1145,"[919482, 672171, 214920]",1,3,plane5,"[364.43042071197414, 202.915857605178, 5.0]",6717.567857,52.086972,5.775554,fit_search_v0.0.1


In [ ]:
##===== Load session from golden mouse =====##
goldenmouse_id = 409828; col = 1; vol = 3
sess = client.load_ophys_session(mouse=goldenmouse_id, column=col, volume=vol)
print(f"Loaded ophys session {sess.get_session_id()} (mouse {sess.get_mouse_id()}, column {sess.get_column_id()}, volume {sess.get_volume_id()})")


In [ ]:
# em_client = V1DDEMClient()